In [1]:
from importlib import reload
import environments.ControlledRangeVariance
import MLE.MLE 

reload(environments.ControlledRangeVariance)
reload(MLE.MLE)

def batchtoonline(samples, seed=45):
    import numpy as np
    
    state = np.random.RandomState(seed)
    
    n = sum(c for c, w, r in samples)
    while n > 0:
        p = np.array([ c for c, w, r in samples ], dtype='float64') / n
        what = state.choice(len(samples), p=p)
        c = min(samples[what][0], 1)
        yield (c, samples[what][1], samples[what][2])
        samples[what] = (samples[what][0] - c, samples[what][1], samples[what][2]) 
        n -= c
    
env = environments.ControlledRangeVariance.ControlledRangeVariance(seed=45, wsupport=[0,2,1000], expwsq=100)
happrox = MLE.MLE.Online.HistApprox(wmin=0, wmax=1000, numbuckets=10)
onlineci = MLE.MLE.Online.CI(wmin=0, wmax=1000, rmin=0, rmax=1, alpha=0.05)
onlinemle = MLE.MLE.Online.MLE(wmin=0, wmax=1000, rmin=0, rmax=1)
_, samples = env.sample(1 << 16)
from pprint import pformat
print(pformat(samples), flush=True)

for n, (c, w, r) in enumerate(batchtoonline(samples)):
    happrox.update(c, w, r)
    onlineci.update(happrox.iterator)
    onlinemle.update(happrox.iterator)
    
    if n & (n - 1) == 0 and n & 0xAAAAAAAA == 0:
        from pprint import pformat
        print(pformat(
                {
                    'n': n,
                    'onlineci': onlineci.getqfunc(),
                    'onlinemle': onlinemle.getqfunc(),
                    'batchmle': MLE.MLE.estimate(happrox.iterator, wmin=0, wmax=1000), 
                    'batchci': MLE.MLE.asymptoticconfidenceinterval(happrox.iterator, wmin=0, wmax=1000)
                }), 
              flush=True)

[(35818, 0, 0), (5658, 2, 0), (24056, 2, 1), (2, 1000, 0), (2, 1000, 1)]
{'batchci': ((0, 1), (None, None)),
 'batchmle': (1.0,
              {'betastar': 1.0000000000000002,
               'num': 1.0000000000000002,
               'qfunc': <function estimate.<locals>.<lambda> at 0x7f54b1000bf8>,
               'vmax': 1.0,
               'vmin': 0.9999999999999999}),
 'n': 0,
 'onlineci': {'betastar': 0.7090212316407816,
              'ci': False,
              'qfunc': <function Online.MLE.getqfunc.<locals>.<lambda> at 0x7f54b1f7dbf8>},
 'onlinemle': {'betastar': 0.7090212316407816,
               'qfunc': <function Online.MLE.getqfunc.<locals>.<lambda> at 0x7f54b1f7dae8>}}
{'batchci': ((-1.000575304586373e-05, 1.0000100056782093),
             ({'betastar': 2.4394412698756992e-09,
               'gammastar': 1.0002833325839528e-05,
               'kappastar': -2.4013937716326954e-10,
               'qfunc': <function asymptoticconfidenceinterval.<locals>.<lambda> at 0x7f54b1007b70>}

{'batchci': ((0.6749275359602328, 0.8005776471032195),
             ({'betastar': 0.0267110489594987,
               'gammastar': 31.76970895527579,
               'kappastar': 0.007925640112325032,
               'qfunc': <function asymptoticconfidenceinterval.<locals>.<lambda> at 0x7f54b1007b70>},
              {'betastar': 0.927346153472718,
               'gammastar': 72.65455134980716,
               'kappastar': 0.017764539872144734,
               'qfunc': <function asymptoticconfidenceinterval.<locals>.<lambda> at 0x7f54b1007048>})),
 'batchmle': (0.7057953819322931,
              {'betastar': 5.125169908914246,
               'num': 4096.999999999982,
               'qfunc': <function estimate.<locals>.<lambda> at 0x7f54b1000d08>,
               'vmax': 0.7057953819322931,
               'vmin': 0.7057953819322931}),
 'n': 4096,
 'onlineci': {'betastar': 0.015884247024729233,
              'ci': True,
              'gammastar': 22.084296700287563,
              'kappastar': 0.